In [1]:
import tensorflow as tf
tf.enable_eager_execution()
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
from bert import modeling
import numpy as np

W0127 18:47:36.258705 139945333634880 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/bert/optimization.py:87: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [2]:
with open('data/counts_1grams.txt') as fopen:
    f = fopen.read().split('\n')[:-1]
    
words = {}
for l in f:
    w, c = l.split('\t')
    c = int(c)
    words[w] = c + words.get(w, 0)

In [3]:
# original from https://github.com/cbaziotis/ekphrasis/blob/master/ekphrasis/classes/spellcorrect.py
# improved it

import re
from collections import Counter

class SpellCorrector:
    """
    The SpellCorrector extends the functionality of the Peter Norvig's
    spell-corrector in http://norvig.com/spell-correct.html
    """
    REGEX_TOKEN = re.compile(r'\b[a-z]{2,}\b')

    def __init__(self, words):
        """
        :param corpus: the statistics from which corpus to use for the spell correction.
        """
        super().__init__()
        self.WORDS = words
        self.N = sum(self.WORDS.values())
              
    @staticmethod
    def tokens(text):
        return REGEX_TOKEN.findall(text.lower())

    def P(self, word):
        """
        Probability of `word`.
        """
        return self.WORDS[word] / self.N

    def most_probable(self, words):
        _known = self.known(words)
        if _known:
            return max(_known, key=self.P)
        else:
            return []

    @staticmethod
    def edit_step(word):
        """
        All edits that are one edit away from `word`.
        """
        letters = 'abcdefghijklmnopqrstuvwxyz'
        splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
        deletes = [L + R[1:] for L, R in splits if R]
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
        replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
        inserts = [L + c + R for L, R in splits for c in letters]
        return set(deletes + transposes + replaces + inserts)

    def edits2(self, word):
        """
        All edits that are two edits away from `word`.
        """
        return (e2 for e1 in self.edit_step(word)
                for e2 in self.edit_step(e1))

    def known(self, words):
        """
        The subset of `words` that appear in the dictionary of WORDS.
        """
        return set(w for w in words if w in self.WORDS)

    def edit_candidates(self, word, assume_wrong=False, fast=True):
        """
        Generate possible spelling corrections for word.
        """

        if fast:
            ttt = self.known(self.edit_step(word)) or {word}
        else:
            ttt = self.known(self.edit_step(word)) or self.known(self.edits2(word)) or {word}
        
        ttt = self.known([word]) | ttt
        return list(ttt)

In [4]:
corrector = SpellCorrector(words)

In [5]:
#possible_states = corrector.edit_candidates('eting')
possible_states = corrector.edit_candidates('gife')
possible_states

['life', 'gift', 'wife', 'gibe', 'give', 'rife']

In [6]:
BERT_VOCAB = 'uncased_L-12_H-768_A-12/vocab.txt'
BERT_INIT_CHKPNT = 'uncased_L-12_H-768_A-12/bert_model.ckpt'
BERT_CONFIG = 'uncased_L-12_H-768_A-12/bert_config.json'

In [7]:
tokenization.validate_case_matches_checkpoint(True,BERT_INIT_CHKPNT)
tokenizer = tokenization.FullTokenizer(
      vocab_file=BERT_VOCAB, do_lower_case=True)

W0127 20:09:54.958088 139945333634880 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/bert/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



In [8]:
#text = 'scientist suggests eting burger can lead to obesity'
text = 'gife me something to eat'
#text_mask = text.replace('eting', '**mask**')
text_mask = text.replace('gife', '**mask**')
text_mask

'**mask** me something to eat'

In [9]:
def tokens_to_masked_ids(tokens, mask_ind):
    masked_tokens = tokens[:]
    masked_tokens[mask_ind] = "[MASK]"
    masked_tokens = ["[CLS]"] + masked_tokens + ["[SEP]"]
    masked_ids = tokenizer.convert_tokens_to_ids(masked_tokens)
    return masked_ids

In [10]:
bert_config = modeling.BertConfig.from_json_file(BERT_CONFIG)

In [43]:
class SpellingModel(tf.keras.Model):
    def __init__(self, X):
        super(SpellingModel, self).__init__()
        self.X = X 
        model = modeling.BertModel(
            config=bert_config,
            is_training=False,
            input_ids=self.X,
            use_one_hot_embeddings=False)
        self.output_layer = model.get_sequence_output()
        self.embedding = model.get_embedding_table()
        
    def call(self, X):
        return self.output_layer
        

In [36]:
tvars = tf.trainable_variables()
(assignment_map, initialized_variable_names
      ) = modeling.get_assignment_map_from_checkpoint(tvars, BERT_INIT_CHKPNT)
assignment_map
initialized_variable_names

{}

In [15]:
replaced_masks = [text_mask.replace('**mask**', state) for state in possible_states]
replaced_masks

['life me something to eat',
 'gift me something to eat',
 'wife me something to eat',
 'gibe me something to eat',
 'give me something to eat',
 'rife me something to eat']

In [16]:
sent = replaced_masks[1]
sent

'gift me something to eat'

In [17]:
tokens = tokenizer.tokenize(sent)
tokens

['gift', 'me', 'something', 'to', 'eat']

In [45]:
input_ids = [tokens_to_masked_ids(tokens, i) for i in range(len(tokens))]

list

In [44]:
spellingModel = SpellingModel(input_ids)

AttributeError: '_ListWrapper' object has no attribute 'name'